In [1]:
import sqlite3
import folium
from folium.plugins import HeatMap
from IPython.display import display
import requests

# Geocoding function using OpenStreetMap Nominatim
def geocode(address):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json",
        "limit": 1
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = float(data[0]["lat"])
            lon = float(data[0]["lon"])
            return lat, lon
    return None, None

# Connect to the SQLite database
conn = sqlite3.connect("BostonHousing-oddyears.db")
cursor = conn.cursor()

# Query the database for the houses in 2022
query = """
    SELECT B.st_num, B.st_name, B.zipcode
    FROM Buildings AS B
    INNER JOIN Assessments AS A ON B.PID = A.PID
    WHERE A.year = 2022
"""
cursor.execute(query)
data = cursor.fetchall()

# Create empty lists to store latitude and longitude values
latitudes = []
longitudes = []

# Geocode the addresses and append coordinates to the lists
for row in data:
    st_num, st_name, zipcode = row

    # Construct the address string
    address = f"{st_num} {st_name}, Boston, {zipcode}"

    # Geocode the address
    lat, lon = geocode(address)

    # Add latitude and longitude to the lists
    if lat and lon:
        latitudes.append(lat)
        longitudes.append(lon)

# Create a folium map centered around Boston
map_boston = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Create a HeatMap layer using the coordinates with smoothing
heat_data = list(zip(latitudes, longitudes))
HeatMap(heat_data, blur=10).add_to(map_boston)  # Adjust the blur parameter to control the smoothing

# Display the map directly in Jupyter Notebook
display(map_boston)


In [10]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('BostonHousing-oddyears.db')

# Fetch data from the Buildings table
query = """
SELECT land_sf, PID, st_name, st_name_suf, st_num, zipcode
FROM Buildings
"""
buildings_data = pd.read_sql_query(query, conn)

# Convert st_num and zipcode to string and fill missing values with an empty string
buildings_data['st_num'] = buildings_data['st_num'].astype(str).fillna('')
buildings_data['zipcode'] = buildings_data['zipcode'].astype(str).fillna('')

# Create a new column for the combined address
buildings_data['address'] = (
    buildings_data['st_num'] + ' ' +
    buildings_data['st_name'] + ' ' +
    buildings_data['st_name_suf'] + ', Boston, MA ' +
    buildings_data['zipcode']
)

# Geocode the addresses to obtain latitude and longitude
geolocator = Nominatim(user_agent="my_geocoder")
buildings_data['location'] = buildings_data['address'].apply(geolocator.geocode)
buildings_data['latitude'] = buildings_data['location'].apply(lambda loc: loc.latitude if loc else None)
buildings_data['longitude'] = buildings_data['location'].apply(lambda loc: loc.longitude if loc else None)

# Update the Buildings table in the database with new columns for latitude and longitude
buildings_data[['latitude', 'longitude', 'PID']].to_sql('Buildings', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=104.0+PUTNAM+ST%2C+Boston%2C+MA+2128.0&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import geopy
from geopy.geocoders import Nominatim
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('BostonHousing-oddyears.db')

# Execute the SQL query
query = """
SELECT b.st_num, b.st_name, b.zipcode, AVG(a.LV) AS avg_land_value, AVG(a.BV) AS avg_building_value
FROM Buildings AS b
JOIN Assessments AS a ON b.PID = a.PID
WHERE a.year = 2022
GROUP BY b.st_num, b.st_name, b.zipcode
"""
data_2022 = pd.read_sql_query(query, conn)

# Create a new column for the combined address
data_2022['address'] = data_2022['st_num'].astype(str) + ' ' + data_2022['st_name'] + ', Boston, MA ' + data_2022['zipcode'].str.replace(',', '')

# Geocode the addresses to obtain latitude and longitude
geolocator = Nominatim(user_agent="my_heatmap")
data_2022['location'] = data_2022['address'].apply(geolocator.geocode)
data_2022['latitude'] = data_2022['location'].apply(lambda loc: loc.latitude if loc else None)
data_2022['longitude'] = data_2022['location'].apply(lambda loc: loc.longitude if loc else None)

# Filter out addresses with missing latitude or longitude
data_2022 = data_2022.dropna(subset=['latitude', 'longitude'])

# Initialize a map centered around Boston
boston_map = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Iterate over the data and add markers to the map
for index, row in data_2022.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['address'],
        popup=f"Land Value: {row['avg_land_value']}, Building Value: {row['avg_building_value']}"
    ).add_to(boston_map)

# Create a HeatMap layer based on the latitudes and longitudes of the addresses
heat_data = data_2022[['latitude', 'longitude']].values
HeatMap(heat_data).add_to(boston_map)

# Visualize the map directly in Jupyter Notebook
boston_map



In [15]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import sqlite3

# Step 1: Retrieve data from the database
connection = sqlite3.connect('BostonHousing-oddyears.db')
query = """
    SELECT st_num, st_name, st_name_suf, BV, LV
    FROM Buildings b
    INNER JOIN Assessments a ON b.PID = a.PID
    LIMIT 100;
"""
df = pd.read_sql_query(query, connection)

# Step 2: Convert 'BV' and 'LV' columns to numeric type
df['BV'] = pd.to_numeric(df['BV'], errors='coerce')
df['LV'] = pd.to_numeric(df['LV'], errors='coerce')

# Step 3: Group by street address and calculate the sum of BV and LV
grouped_df = df.groupby(['st_num', 'st_name', 'st_name_suf']).agg({'BV': 'sum', 'LV': 'sum'}).reset_index()

# Step 4: Geocode street addresses
geolocator = Nominatim(user_agent='heat_map')
grouped_df['address'] = grouped_df['st_num'].astype(str) + ' ' + grouped_df['st_name'] + ' ' + grouped_df['st_name_suf']
grouped_df['location'] = grouped_df['address'].apply(geolocator.geocode)
grouped_df['point'] = grouped_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Step 5: Create the heat map
heat_map = folium.Map(location=[42.3601, -71.0589], zoom_start=12)  # Set the initial map location
heat_data = [[point[0], point[1], bv+lv] for point, bv, lv in zip(grouped_df['point'], grouped_df['BV'], grouped_df['LV']) if point is not None]
folium.plugins.HeatMap(heat_data).add_to(heat_map)

# Step 6: Save the heat map as an HTML file
heat_map.save('building_heat_map.html')




In [10]:
import pandas as pd
import folium
import sqlite3

# Step 1: Retrieve data from the database
connection = sqlite3.connect('BostonHousing-oddyears.db')
query = """
    SELECT zipcode, BV, LV
    FROM Buildings b
    INNER JOIN Assessments a ON b.PID = a.PID;
"""
df = pd.read_sql_query(query, connection)

# Step 2: Convert 'BV' and 'LV' columns to numeric type
df['BV'] = pd.to_numeric(df['BV'], errors='coerce')
df['LV'] = pd.to_numeric(df['LV'], errors='coerce')

# Step 3: Group by zipcode and calculate the average of BV+LV
grouped_df = df.groupby('zipcode').agg({'BV': 'mean', 'LV': 'mean'}).reset_index()

# Step 4: Calculate the difference from the average cost
grouped_df['Difference'] = grouped_df['BV'] - grouped_df['LV']

# Step 5: Create the map with markers
map_center = [42.3601, -71.0589]  # Center coordinates of the map
map_zoom = 12  # Zoom level of the map

# Generate the map
m = folium.Map(location=map_center, zoom_start=map_zoom)

# Add markers to the map
for _, row in grouped_df.iterrows():
    zip_code = row['zipcode']
    difference = row['Difference']

    # Adjust marker color and size based on the difference from the average cost
    if difference > 0:
        color = 'green'
        size = 'large'
    elif difference < 0:
        color = 'red'
        size = 'medium'
    else:
        color = 'yellow'
        size = 'small'

    folium.Marker(
        location=[0, 0],  # Replace with the actual coordinates of the zip code
        popup=f"Zip Code: {zip_code}<br>Difference: {difference}",
        icon=folium.Icon(color=color, icon='cloud', prefix='fa', icon_color='white', icon_size=size)
    ).add_to(m)

# Save the map as an HTML file
m.save('zipcode_marker_map.html')



In [16]:
import folium
import pandas as pd

# Load your data into a pandas DataFrame
# Assuming you have a DataFrame named 'houses' with columns 'latitude', 'longitude', 'zipcode', and 'price'

# Group the data by zip code and calculate the mean price for each group
grouped_data = houses.groupby('zipcode').mean().reset_index()

# Create a map centered around Boston
boston_map = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Define a color scale for the price
color_scale = folium.LinearColormap(['green', 'yellow', 'red'], vmin=grouped_data['price'].min(), vmax=grouped_data['price'].max())

# Iterate over the grouped data and add markers to the map
for _, row in grouped_data.iterrows():
    zipcode = str(row['zipcode'])
    price = row['price']
    location = [row['latitude'], row['longitude']]
    folium.CircleMarker(
        location=location,
        radius=5,
        color=color_scale(price),
        fill=True,
        fill_color=color_scale(price),
        fill_opacity=0.7,
        popup=f"Zipcode: {zipcode}<br>Price: {price:.2f}"
    ).add_to(boston_map)

# Add the color scale to the map
color_scale.caption = 'Price Scale'
color_scale.add_to(boston_map)

# Save the map as an HTML file
boston_map.save('boston_map.html')



NameError: name 'houses' is not defined